In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import csv
import os
import re

# Specify the path to the folder containing the input text files
folder_path = '/content/drive/MyDrive/QA/output'

# Specify the path for the output CSV file
output_csv_path = '/content/drive/MyDrive/QA/file.csv'

# Function to extract questions and answers from a text file
def extract_questions_answers(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    questions = re.findall(r'\*\*Question \d+:\*\*(.*?)\*\*Answer:\*\*(.*?)(?=\*\*Question \d+:\*\*|\Z)', text, re.DOTALL)
    formatted_data = [[file_path, q.strip(), a.strip()] for q, a in questions]
    return formatted_data

# Initialize an empty list to store all extracted data
all_data = []

# Loop through each file in the folder
for filename in sorted(os.listdir(folder_path)):  # Sort files by filename
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        # Extract questions and answers from the current file
        extracted_data = extract_questions_answers(file_path)
        # Append extracted data to the list
        all_data.extend(extracted_data)

# Write all extracted data into the output CSV file, sorted by filename
with open(output_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)

    # Write the header row
    csv_writer.writerow(['Question', 'Answer'])

    # Sort data by filename before writing to CSV
    sorted_data = sorted(all_data, key=lambda x: os.path.basename(x[0]))  # Sort by filename

    # Write each row of data (filename, question, answer)
    for row in sorted_data:
        csv_writer.writerow(row[1:])  # Exclude file_path in the CSV output

print(f"CSV file '{output_csv_path}' has been created successfully with data from all text files in the folder.")


In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" -q
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
dataset = pd.read_csv("/content/file.csv")
dataset.head(6)

,Question,Answer
0,"শিশু আইন, ২০১৩ অনুযায়ী, শিশুকে নিরাপদ হেফাজতে ...",শিশুকে নিরাপদ হেফাজতে রাখার বিষয়টিকে সর্বশেষ প...
1,নিরাপদ হেফাজতে রক্ষিত শিশুকে কত দ্রুত বিকল্পপন...,সম্ভাব্য দ্রুততম সময়ের মধ্যে।
2,শিশুকে নিরাপদ হেফাজতে রাখার একান্ত প্রয়োজন হলে...,শিশু-আদালত সংশ্লিষ্ট শিশুকে উক্ত আদালত হইতে যু...
3,"শিশু আইন, ২০১৩ অনুযায়ী, শিশুদের সাক্ষ্য গ্রহণে...",শিশুর জন্য সরল ও বোধগম্য ভাষা।
4,শিশুর সাক্ষ্য ব্যাখ্যা করার জন্য কীভাবে সহায়তা...,আদালত বিনা খরচে শিশুকে একজন ব্যাখ্যাকারী প্রদা...
5,"শিশু আইন, ২০১৩-এর এই বিধানের উদ্দেশ্য কী?",শিশুদের আইনি প্রক্রিয়া বোঝার এবং তাদের সাক্ষ্য...


In [ ]:
dataset.shape

(132, 2)

In [ ]:
# Split the data into training, validation, and test sets
train, X_temp = train_test_split(dataset, test_size=0.3, random_state=42)
validation, test = train_test_split(X_temp, test_size=0.10, random_state=42)

In [ ]:
train = train.reset_index(drop=True)
validation = validation.reset_index(drop=True)
test = test.reset_index(drop=True)

In [ ]:
train.head()

,Question,Answer
0,কোন শিশুকে কারাদণ্ড দেওয়ার ক্ষেত্রে শিশু আদালত...,যদি শিশুটি এত বেশি অবাধ্য বা ভ্রষ্ট চরিত্রের হ...
1,"সড়ক রক্ষণাবেক্ষণ তহবিল বোর্ড আইন, ২০১৩ অনুযায়ী...","বোর্ড, উহার কার্যাবলী দক্ষতার সহিত সম্পাদনে সহ..."
2,"বাংলা একাডেমি আইন, ২০১৩ অনুযায়ী, জীবনসদস্য ও স...","ভাষা, সাহিত্য, সংস্কৃতি, বিজ্ঞান, শিল্পকলা, সা..."
3,একাডেমি বিশ্ববিদ্যালয় বা প্রতিষ্ঠানের সহায়তা গ...,নির্বাহী পরিষদের।
4,"শিশু আইন, ২০১৩ অনুযায়ী, শিশু-আদালত কর্তৃক আদেশ...","দণ্ডবিধিতে ব্যবহৃত ‘অপরাধী’, ‘দণ্ডিত’ বা ‘দণ্ড..."


In [ ]:
# Create a datasets.Dataset object
train = Dataset.from_dict(train)
validation = Dataset.from_dict(validation)
test = Dataset.from_dict(test)
# Print the dataset to check the format
print("Train Dataset : ",train)
print("Validation Dataset : ", validation)
print("Test Dataset : ", test)

Train Dataset :  Dataset({
    features: ['Question', 'Answer'],
    num_rows: 92
})
Validation Dataset :  Dataset({
    features: ['Question', 'Answer'],
    num_rows: 36
})
Test Dataset :  Dataset({
    features: ['Question', 'Answer'],
    num_rows: 4
})


In [ ]:
# Create DatasetDict
from datasets import DatasetDict
dataset = DatasetDict({
    'validation': validation,
    'test': test,
    'train': train
})
print(dataset)

DatasetDict({
    validation: Dataset({
        features: ['Question', 'Answer'],
        num_rows: 36
    })
    test: Dataset({
        features: ['Question', 'Answer'],
        num_rows: 4
    })
    train: Dataset({
        features: ['Question', 'Answer'],
        num_rows: 92
    })
})


In [ ]:
dataset['train']['Answer'][91]

'সাধারণ পরিষদের উপর।'

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "unsloth/llama-3-8b-bnb-4bit",
    model_name = "meta-llama/Meta-Llama-3-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
    token = "",
)

Unsloth: You passed in `meta-llama/Meta-Llama-3-8B` and `load_in_4bit = True`.
We shall load `unsloth/llama-3-8b-bnb-4bit` for 4x faster loading.


==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## **We now add LoRA adapters so we only need to update 1 to 10% of all parameters!**

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


**Data Preparation**
Remember to add the EOS_TOKEN to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the ChatML template for ShareGPT datasets, try our conversational notebook.

For text completions like novel writing, try this notebook.**

In [ ]:
# alpaca_prompt = """
# ### Instruction:
# {}

# ### Input:
# {}

# ### Response:
# {}"""

# EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
# def formatting_prompts_func(examples):
#     instructions = "Please provide a detailed answer to the following question"
#     inputs = examples["Question"]
#     outputs = examples["Answer"]
#     texts = []
#     for instruction, input, output in zip(instructions, inputs, outputs):
#         # Must add EOS_TOKEN, otherwise your generation will go on forever!
#         text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
#         texts.append(text)
#     return {"text" : texts}


# dataset = dataset.map(formatting_prompts_func, batched = True,)


alpaca_prompt = """
### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(example):
    # Retrieve question and answer from the example
    instruction = "Please provide a detailed answer to the following question"
    question = example["Question"]
    answer = example["Answer"]

    # Check the structure and content of the example
    print(f"Question: {question}")
    print(f"Answer: {answer}")

    # Construct the formatted prompt text
    prompt_text = alpaca_prompt.format(instruction, question, answer) + EOS_TOKEN

    # Return the formatted prompt text as a dictionary
    return {"text": prompt_text}

# Assuming 'dataset' is your dataset object
dataset = dataset.map(formatting_prompts_func)

# Now check the dataset and ensure that it has been transformed correctly


Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Question: বাংলা একাডেমির মুখ্য প্রতিষ্ঠান হিসাবে কাজ করার ক্ষেত্রে এর প্রাথমিক লক্ষ্য কী?
Answer: বাংলা ভাষার প্রমিতকরণ এবং উৎকর্ষ সাধন।
Question: কমিটি গঠনের উদ্দেশ্য কী?
Answer: বোর্ডের কার্যাবলী দক্ষতার সহিত সম্পাদনে সহায়তা করা।
Question: শিশু আইন, ২০১৩ অনুযায়ী, আইনের সঙ্গে সংঘাতে জড়িত শিশুকে শিশু আদালতে হাজির করার কত দিনের মধ্যে সামাজিক অনুসন্ধান প্রতিবেদন দাখিল করতে হবে?
Answer: অনধিক ২১ (একুশ) দিনের মধ্যে।
Question: বাংলা একাডেমি পুরস্কার বা সম্মাননা প্রদানের ক্ষেত্রে কোন নিয়মাবলী অনুসরণ করে?
Answer: বিধি অনুযায়ী।
Question: কোনো সদস্যের সদস্যপদ বাতিল হলে বা পদত্যাগ করলে কী ঘটে?
Answer: সদস্যপদ বাতিল হলে বা পদত্যাগ করলে তিনি পুনরায় সদস্য হতে পারবেন না।
Question: বাজেট বিবরণীতে কী থাকবে?
Answer: সম্ভাব্য আয়-ব্যয় এবং সরকারের কাছ থেকে বোর্ডের প্রয়োজনীয় অর্থের পরিমাণ।
Question: বোর্ডের ক্ষমতা অর্পণের ক্ষেত্রে কী বিবেচনা করা হয়?
Answer: প্রয়োজনবোধ।
Question: শিশু আদালত কোন শিশুকে আটকাদেশের পরিবর্তে কী দিতে পারে?
Answer: যথাযথ সতর্কীকরণের পর খালাস প্রদানের অথবা সদাচারণের জন্য প্রবেশনে ম

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Question: সরকার বোর্ডের কার্যক্রম সম্পর্কে কী করতে পারে?
Answer: সরকার যে কোন সময়ে বোর্ডের কর্মকাণ্ড অথবা যে কোন প্রকার অভিযোগের বিষয়ে তদন্তের নির্দেশ দিতে পারে।
Question: সড়ক রক্ষণাবেক্ষণ তহবিল বোর্ড আইন, ২০১৩ অনুযায়ী, পরিচালনা পরিষদের সভা আহ্বানের দায়িত্ব কার?
Answer: পরিচালনা পরিষদের সদস্য-সচিব, চেয়ারম্যানের সম্মতিক্রমে।
Question: বাংলা একাডেমি আইন, ২০১৩ অনুযায়ী, সাধারণ পরিষদের কী কী দায়িত্ব রয়েছে?
Answer: বাংলা একাডেমি আইন, ২০১৩ এই বিষয়ে কোনো তথ্য প্রদান করে না।
Question: বাৎসরিক প্রতিবেদন কার কাছে উপস্থাপন করা হবে?
Answer: নির্বাহী পরিষদের নিকট।


Map:   0%|          | 0/92 [00:00<?, ? examples/s]

Question: কোন শিশুকে কারাদণ্ড দেওয়ার ক্ষেত্রে শিশু আদালতের বিশেষ বিবেচনা কী?
Answer: যদি শিশুটি এত বেশি অবাধ্য বা ভ্রষ্ট চরিত্রের হয় যে তাকে প্রত্যয়িত প্রতিষ্ঠানে প্রেরণ করা যায় না এবং অন্যান্য আইনানুগ পন্থায় মামলাটির সুরাহা করা যায় না।
Question: সড়ক রক্ষণাবেক্ষণ তহবিল বোর্ড আইন, ২০১৩ অনুযায়ী, বোর্ড কোন কাজের জন্য কমিটি গঠন করতে পারে?
Answer: বোর্ড, উহার কার্যাবলী দক্ষতার সহিত সম্পাদনে সহায়তা করিবার জন্য কমিটি গঠন করিতে পারিবে।
Question: বাংলা একাডেমি আইন, ২০১৩ অনুযায়ী, জীবনসদস্য ও সদস্য হওয়ার জন্য কোন শর্তগুলি পূরণ করতে হবে?
Answer: ভাষা, সাহিত্য, সংস্কৃতি, বিজ্ঞান, শিল্পকলা, সাংবাদিকতা, সামাজিক বিজ্ঞান অথবা জ্ঞানের বিশেষ ক্ষেত্রে বিশেষ অবদান রাখতে হবে এবং বয়স কমপক্ষে ৩০ বছর হতে হবে।
Question: একাডেমি বিশ্ববিদ্যালয় বা প্রতিষ্ঠানের সহায়তা গ্রহণের আগে কার অনুমোদন নিতে হবে?
Answer: নির্বাহী পরিষদের।
Question: শিশু আইন, ২০১৩ অনুযায়ী, শিশু-আদালত কর্তৃক আদেশ প্রদানের ক্ষেত্রে কোন শব্দগুলি ব্যবহার করা যাবে না?
Answer: দণ্ডবিধিতে ব্যবহৃত ‘অপরাধী’, ‘দণ্ডিত’ বা ‘দণ্ডাদেশ’ শব্দসমূহ।
Question: বা

In [ ]:
dataset["train"][1]

{'Question': 'সড়ক রক্ষণাবেক্ষণ তহবিল বোর্ড আইন, ২০১৩ অনুযায়ী, বোর্ড কোন কাজের জন্য কমিটি গঠন করতে পারে?',
 'Answer': 'বোর্ড, উহার কার্যাবলী দক্ষতার সহিত সম্পাদনে সহায়তা করিবার জন্য কমিটি গঠন করিতে পারিবে।',
 'text': '\n### Instruction:\nPlease provide a detailed answer to the following question\n\n### Input:\nসড়ক রক্ষণাবেক্ষণ তহবিল বোর্ড আইন, ২০১৩ অনুযায়ী, বোর্ড কোন কাজের জন্য কমিটি গঠন করতে পারে?\n\n### Response:\nবোর্ড, উহার কার্যাবলী দক্ষতার সহিত সম্পাদনে সহায়তা করিবার জন্য কমিটি গঠন করিতে পারিবে।<|end_of_text|>'}

# Train the model
We do 60 steps to speed things up, but you can set num_train_epochs=1 for a full run, and turn off max_steps=None.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    eval_dataset = dataset["validation"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 40,
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        evaluation_strategy='epoch',
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer_stats = trainer.train()

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/92 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/36 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 92 | Num Epochs = 40
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 440
 "-____-"     Number of trainable parameters = 41,943,040


Epoch,Training Loss,Validation Loss
0,0.513500,0.466764
2,0.224600,0.391904
4,0.089200,0.488527
6,0.066500,0.524035
8,0.035200,0.594603
10,0.032900,0.601779
12,0.024100,0.635921
14,0.026100,0.643747
16,0.031600,0.680066
18,0.037500,0.690096


Epoch,Training Loss,Validation Loss
0,0.513500,0.466764
2,0.224600,0.391904
4,0.089200,0.488527
6,0.066500,0.524035
8,0.035200,0.594603
10,0.032900,0.601779
12,0.024100,0.635921
14,0.026100,0.643747
16,0.031600,0.680066
18,0.037500,0.690096


# Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Please provide a detailed answer to the following question", # instruction
        "বাংলা একাডেমি আইন, ২০১৩ অনুযায়ী, সদস্যপদ বাতিল কারণ?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>
### Instruction:
Please provide a detailed answer to the following question

### Input:
বাংলা একাডেমি আইন, ২০১৩ অনুযায়ী, সদস্যপদ বাতিল কারণ?

### Response:
বাংলা একাডেমি আইন, ২০১৩ অনুযায়ী, সদস্যপদ বাতিল কারণ যদি নিম্নলিখিত কোন কারণে হয়, তদ্ব্যতীত একাডেমি প্রত্যয়িত করিতে পারিবে যে 

Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's push_to_hub for an online save or save_pretrained for a local save.

## [NOTE] This ONLY saves the LoRA adapters, and not the full model.

In [ ]:
# !pip install huggingface_hub --q
# !huggingface-cli login

In [ ]:
# # model.save_pretrained("Bangla-Text-Question-Answer") # Local saving
# model.push_to_hub("Bikas0/Bangla-Text-Question-Answer", token = "") # Online saving

In [ ]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/MyDrive/QA/Bangla-Text-Question-Answer", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Please provide a detailed answer to the following question", # instruction
        "সাধারণ পরিষদের সভা কোথায় অনুষ্ঠিত হয়?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)

In [ ]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Please provide a detailed answer to the following question", # instruction
        "সাধারণ পরিষদের সভা কোথায় অনুষ্ঠিত হয়?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 2048, use_cache = True)
tokenizer.batch_decode(outputs)

You can also use Hugging Face's AutoModelForPeftCausalLM. Only use this if you do not have unsloth installed. It can be hopelessly slow, since 4bit model downloading is not supported, and Unsloth's inference is 2x faster.

In [ ]:
# I highly do NOT suggest - use Unsloth if possible
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer
model = AutoPeftModelForCausalLM.from_pretrained(
    "lora_model", # YOUR MODEL YOU USED FOR TRAINING
    load_in_4bit = load_in_4bit,
)
tokenizer = AutoTokenizer.from_pretrained("lora_model")

# Saving to float16 for VLLM
We also support saving to float16 directly. Select merged_16bit for float16 or merged_4bit for int4. We also allow lora adapters as a fallback. Use push_to_hub_merged to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

# GGUF / llama.cpp Conversion
To save to GGUF / llama.cpp, we support it natively now! We clone llama.cpp and we default save it to q8_0. We allow all methods like q4_k_m. Use save_pretrained_gguf for local saving and push_to_hub_gguf for uploading to HF.

*   q8_0 - Fast conversion. High resource use, but generally
*   q4_k_m - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.

*   q5_k_m - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.


In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")